In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir  = '../data/'
implan_dir = data_dir + 'implan/'

## Import Data

In [3]:
state_ind_sum_files = [x for x in os.walk(implan_dir)][0][2]
state_ind_sum_files = [x for x in state_ind_sum_files if 'Summary' in x]

In [4]:
state_df_list = []

for file in state_ind_sum_files:
    
    state_df = pd.read_csv(implan_dir + file, skiprows = 1)

    state_df['State Name'] = (' ').join(file.split(' ')[0:-2])
    state_df['State Name'] = state_df['State Name'].str.upper() 

    state_df_list.append(state_df)

In [5]:
state_ind_sum = pd.concat(state_df_list, ignore_index=True)

### Keys

In [6]:
# MSN Codes Key
msn_codes_file_loc = data_dir + 'keys/MSN_codes.csv'
msn_codes_key = pd.read_csv(msn_codes_file_loc)

# State FIPS Codes
state_fips_file_loc = data_dir + 'keys/state_FIPS.csv'
state_fips_key = pd.read_csv(state_fips_file_loc)
state_fips_indicators = ['State Abbreviation', 'State Name']

# State NCDC Codes
state_ncdc_file_loc = data_dir + 'keys/state_NCDC_codes.csv'
state_ncdc_key = pd.read_csv(state_ncdc_file_loc)
state_ncdc_key['State Name'] = state_ncdc_key['State Name'].str.upper()

# Merged State Codes Key
state_codes_key = state_ncdc_key.merge(state_fips_key)

# Month Key
month_key = dict(zip(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], range(1,13)))

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


## Clean Data

In [7]:
# Add input costs
state_ind_sum['TotalInputCosts'] = state_ind_sum['Output'] - state_ind_sum['TotalValueAdded'] 

# Pivot Industry Codes
state_ind_sum = state_ind_sum[['IndustryCode', 'State Name', 'TotalInputCosts']]
state_ind_sum = state_ind_sum.pivot(index = 'State Name', columns = 'IndustryCode', values = 'TotalInputCosts').reset_index()

# Grab relevant industry columns
state_ind_sum = state_ind_sum[['State Name'] + list(range(41,48))].rename(columns = {41: 'hydro_tot_input_costs', 42: 'ff_tot_input_costs', 43: 'nuclear_tot_input_costs',
                                                                    44: 'solar_tot_input_costs', 45: 'wind_tot_input_costs', 46: 'geothermal_tot_input_costs',
                                                                    47: 'biomass_tot_input_costs'})

state_ind_sum.columns = pd.Series(state_ind_sum.columns).values

# Add months
state_ind_sum_list = []

for month in range(1,13):
    
    temp_df = state_ind_sum.copy()
    temp_df['month'] = month
    
    state_ind_sum_list.append(temp_df)
    
state_ind_sum = pd.concat(state_ind_sum_list)

In [8]:
state_ind_sum.head()

,State Name,hydro_tot_input_costs,ff_tot_input_costs,nuclear_tot_input_costs,solar_tot_input_costs,wind_tot_input_costs,geothermal_tot_input_costs,biomass_tot_input_costs,month
0,ALABAMA,6.560774e+07,2.463815e+09,6.720215e+08,0.000000e+00,7.708770e+06,0.000000e+00,9.066061e+07,1
1,ALASKA,8.507501e+07,3.938253e+08,0.000000e+00,0.000000e+00,5.515352e+06,0.000000e+00,1.028872e+07,1
2,ARIZONA,0.000000e+00,1.470722e+09,3.000149e+05,5.558335e+07,1.911451e+05,0.000000e+00,3.489155e+05,1
3,ARKANSAS,2.236884e+06,3.072770e+08,4.102857e+08,0.000000e+00,0.000000e+00,6.248546e+06,0.000000e+00,1
4,CALIFORNIA,5.134468e+08,8.126158e+09,1.495902e+09,1.551753e+08,2.043318e+08,2.384203e+08,4.840587e+08,1


## Merge Data

### Add FIPS codes

In [9]:
state_ind_sum = state_ind_sum.merge(state_fips_key).drop(state_fips_indicators, axis = 1)

# Export Data

In [10]:
state_ind_sum.to_csv(data_dir + 'processed/implan_data.csv', index = False)